In [ ]:
import pandas as pd

df = pd.read_csv('A2 - A2.csv')
df

In [ ]:
df=df.dropna()

In [ ]:
from textblob import TextBlob

# Define a function to classify sentiment
def classify_sentiment(text, threshold=0.4):
    analysis = TextBlob(text)
    sentiment_score = analysis.sentiment.polarity
    if sentiment_score >= threshold:
        return 1  # Good experience
    else:
        return 0  # Bad experience

# Assuming 'reviews' is a DataFrame containing the 'review' column
# Replace 'reviews' with the name of your DataFrame and 'review' with the actual column name

# Apply sentiment classification to each review
df['Sentiment'] = df['Review'].apply(lambda x: classify_sentiment(x))

# Now 'sentiment' column contains binary values (0 or 1) indicating bad or good experience


In [ ]:
df

In [ ]:
y = df['Sentiment']
y

In [ ]:
X = df.drop(['ID'], axis=1)
X

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
#from deep_translator import GoogleTranslator

ps = PorterStemmer()

emoji_dictionary = {'👍': 'thumbs_up','❤️':'red_heart','😞':'sad_face',"😍":"love","🤬":"Angry","😊":"Happy ","💝":"Heart","🤑":"Money"}


#old=pd.read_csv("reviews_data - reviews_data.csv")
WithoutStop=[]
lowercase=[]
panctuations=[".",";",",","?","!",":"]
StopWords=["is", "was","this","a","it","it's","me","and","to","I","for","at","of","you","better"]
WithoutPan=[]
Translated=[]
Noemoji=[]
tokens=[]
Step=[]
lemm=[]
for i in list(df["Review"]):
  lowercase.append(i.lower())
  res=""
  for j in lowercase[-1]:
    if j not in panctuations:res=res+j
    else:res=res+" "
  res = res.split()
  tokens.append(res)
  WithoutPan.append(res)
  res=""
  Stemmed=""
  trans=""
  Lemmetised=""
  for k in tokens[-1]:
    if k not in StopWords and k not in panctuations:
      Stemmed=Stemmed+ps.stem(k)+" "
      Lemmetised=Lemmetised+lemmatizer.lemmatize(k,pos="a")+" "
      # trans=trans+str(GoogleTranslator(source='auto', target='en').translate(k))
      res=res+k+" "
    else:
      res=res+" "

  new_text = ""
  for k in i:
      if k in emoji_dictionary:
          new_text +=  emoji_dictionary[k]
      else:
          new_text += k

  Step.append(Stemmed)
  lemm.append(Lemmetised)
  WithoutStop.append(res)
  Noemoji.append(new_text)



#old.drop(['Name', 'Vote1', 'Vote2',"Title"], axis=1, inplace=True)
df["lowercase"]=lowercase
df["tokens"]=tokens
df["Stopped"]=WithoutStop
df["Pan"]=WithoutPan
df["Noemoji"]=Noemoji
df["Stemm"]=Step
df["lemmatisation"]=lemm

In [ ]:
df

In [ ]:
X_test

In [ ]:

#checking the code

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report ,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

#lr = LogisticRegression()
#lr.fit(X_train, y_train)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # TF-IDF vectorization
    ('clf', LogisticRegression())  # Logistic Regression model
])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from itertools import chain

# Flatten the list of lists and convert each token to lowercase
df['tokens_lower'] = [' '.join(map(str.lower, doc)) for doc in df['tokens']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tokens_lower'], y, test_size=0.2, random_state=42)

# Fit the TF-IDF vectorizer to the original text data and transform the training and testing data
tfidf = TfidfVectorizer()
tfidf.fit(X_train)  # Fit to the training data
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train the logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict the labels for the testing data
y_pred = model.predict(X_test_tfidf)




In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))